# 마켓 코드 가져오기

참고: [마켓 코드 조회 페이지](https://docs.upbit.com/reference/%EB%A7%88%EC%BC%93-%EC%BD%94%EB%93%9C-%EC%A1%B0%ED%9A%8C)

#### API 사용하기 위한 라이브러리 설치

- 우선 API를 사용하기 위해 `requests`를 설치합니다   
- 아마도 이미 설치되어 있을 것이기 때문에 코드 셀 실행했을 때 `Requirement already satisfied` 문구가 뜰 것입니다

In [1]:
!pip install requests

#### API로 마켓 코드 조회하기

- [마켓 코드 조회 페이지](https://docs.upbit.com/reference/%EB%A7%88%EC%BC%93-%EC%BD%94%EB%93%9C-%EC%A1%B0%ED%9A%8C)에서 우측의 REQUEST를 보면 API를 사용하기 위한 코드가 그대로 작성되어 있습니다   
- 이 코드를 아래 코드 셀에 복사, 붙여넣기 후 실행해봅시다

In [2]:
import requests

url = "https://api.upbit.com/v1/market/all?isDetails=true"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

[{"market_warning":"NONE","market":"KRW-BTC","korean_name":"비트코인","english_name":"Bitcoin"},{"market_warning":"NONE","market":"KRW-ETH","korean_name":"이더리움","english_name":"Ethereum"},{"market_warning":"NONE","market":"BTC-ETH","korean_name":"이더리움","english_name":"Ethereum"},{"market_warning":"NONE","market":"BTC-XRP","korean_name":"리플","english_name":"Ripple"},{"market_warning":"NONE","market":"BTC-ETC","korean_name":"이더리움클래식","english_name":"Ethereum Classic"},{"market_warning":"NONE","market":"BTC-CVC","korean_name":"시빅","english_name":"Civic"},{"market_warning":"NONE","market":"BTC-DGB","korean_name":"디지바이트","english_name":"DigiByte"},{"market_warning":"NONE","market":"BTC-SC","korean_name":"시아코인","english_name":"Siacoin"},{"market_warning":"NONE","market":"BTC-SNT","korean_name":"스테이터스네트워크토큰","english_name":"Status Network Token"},{"market_warning":"NONE","market":"BTC-WAVES","korean_name":"웨이브","english_name":"Waves"},{"market_warning":"NONE","market":"BTC-NMR","korean_name":"뉴메레

- 출력 결과를 확인해보니 리스트 안에 여러개의 딕셔너리가 들어가 있는 형태입니다   
- 이를 json 형태라 부르며 많은 API 들이 데이터를 json 형태로 전달해줍니다

json을 알아보기 쉽게, 조작하기 쉽게 DataFrame 형태로 변환합니다

In [7]:
import pandas as pd
import json

market_code_df = pd.DataFrame(json.loads(response.text))
market_code_df

,market_warning,market,korean_name,english_name
0,NONE,KRW-BTC,비트코인,Bitcoin
1,NONE,KRW-ETH,이더리움,Ethereum
2,NONE,BTC-ETH,이더리움,Ethereum
3,NONE,BTC-XRP,리플,Ripple
4,NONE,BTC-ETC,이더리움클래식,Ethereum Classic
...,...,...,...,...
299,NONE,BTC-ID,스페이스아이디,SPACE ID
300,NONE,BTC-AXL,엑셀라,Axelar
301,NONE,KRW-CTC,크레딧코인,Creditcoin
302,NONE,KRW-ASTR,아스타,Astar


- market_warning 열은 필요 없어 보입니다
- english_name도 korean_name과 같은 의미를 가지므로 필요 없어 보입니다
- 필요 없어 보이는 열들을 삭제합니다

In [8]:
market_code_df = market_code_df[["market", "korean_name"]]
market_code_df

,market,korean_name
0,KRW-BTC,비트코인
1,KRW-ETH,이더리움
2,BTC-ETH,이더리움
3,BTC-XRP,리플
4,BTC-ETC,이더리움클래식
...,...,...
299,BTC-ID,스페이스아이디
300,BTC-AXL,엑셀라
301,KRW-CTC,크레딧코인
302,KRW-ASTR,아스타


- market열을 보면 `KRW-BTC`, `BTC-ETH` 식으로 되어 있습니다
- 앞의 단어는 어떤 화폐로 거래할지, 뒤의 단어는 어떤 화폐를 거래할지 대한 정보입니다
- `KRW-BTC`은 원화로 비트코인을 사겠다라는 뜻입니다
- 일반적으로 원화를 사용하여 암호화폐를 거래하므로 market의 값 중에서 KRW가 존재하는 row만 추출하겠습니다

In [10]:
market_code_df = market_code_df[market_code_df.market.str.contains("KRW")]
market_code_df = market_code_df.reset_index(drop=True)
market_code_df

,market,korean_name
0,KRW-BTC,비트코인
1,KRW-ETH,이더리움
2,KRW-NEO,네오
3,KRW-MTL,메탈
4,KRW-XRP,리플
...,...,...
113,KRW-IMX,이뮤터블엑스
114,KRW-SEI,세이
115,KRW-MINA,미나
116,KRW-CTC,크레딧코인


- 304개의 행에서 118개의 행으로 줄었습니다

---

# 시세 캔들 조회 해보기

참고: [시세 주(Week)캔들 조회 페이지](https://docs.upbit.com/reference/%EC%A3%BCweek-%EC%BA%94%EB%93%A4-1)

우선 페이지에서 제시한 예시 코드를 그대로 복사 붙여넣기하여 실행해봅시다

In [11]:
url = "https://api.upbit.com/v1/candles/weeks?count=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"error":{"name":400,"message":"Missing request parameter error. Check the required parameters!"}}


- 에러가 출력된 것을 볼 수 있습니다

#### KRW-BTC 캔들 정보 가져오기

- 캔들 조회 페이지를 조금 내려보면 `QUERY PARAMS`의 `market` 부분에 `required`라고 되어 있습니다
- 해당 API를 사용하기 위해선 `market` 정보를 꼭 넘겨줘야 한다는 뜻입니다
- 해당 부분에 `KRW-BTC`를 입력 후 페이지 우측의 REQUEST 코드를 다시 `복사 붙여넣기 실행`해봅시다

In [12]:
url = "https://api.upbit.com/v1/candles/weeks?market=KRW-BTC&count=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

[{"market":"KRW-BTC","candle_date_time_utc":"2024-01-15T00:00:00","candle_date_time_kst":"2024-01-15T09:00:00","opening_price":57380000.00000000,"high_price":59000000.00000000,"low_price":57332000.00000000,"trade_price":58601000.00000000,"timestamp":1705339236178,"candle_acc_trade_price":194583893366.37892000,"candle_acc_trade_volume":3334.13067159,"first_day_of_period":"2024-01-15"}]


- url의 중간에 `market=KRW-BTC&`이 새로 추가된 것을 볼 수 있습니다
- 이번에도 json 형식으로 데이터를 전달 받았습니다
- 다시 DataFrame 형태로 바꿔봅시다

In [13]:
btc_df = pd.DataFrame(json.loads(response.text))
btc_df

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,first_day_of_period
0,KRW-BTC,2024-01-15T00:00:00,2024-01-15T09:00:00,57380000.0,59000000.0,57332000.0,58601000.0,1705339236178,1.945839e+11,3334.130672,2024-01-15


- 하나의 데이터만 가져온 것을 볼 수 있습니다

#### 캔들 여러개 가져오기

- `QUERY PARAMS`을 다시 보면 `count`에 넘긴 값 만큼 캔들을 여러개 가져온다 합니다
- `count`에 200 입력 후 다시 코드를 `복사 붙여넣기 실행` 해봅니다

In [15]:
url = "https://api.upbit.com/v1/candles/weeks?market=KRW-BTC&count=200"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

[{"market":"KRW-BTC","candle_date_time_utc":"2024-01-15T00:00:00","candle_date_time_kst":"2024-01-15T09:00:00","opening_price":57380000.00000000,"high_price":59000000.00000000,"low_price":57332000.00000000,"trade_price":58620000.00000000,"timestamp":1705339494255,"candle_acc_trade_price":194853385877.76963000,"candle_acc_trade_volume":3338.72927021,"first_day_of_period":"2024-01-15"},{"market":"KRW-BTC","candle_date_time_utc":"2024-01-08T00:00:00","candle_date_time_kst":"2024-01-08T09:00:00","opening_price":59105000.00000000,"high_price":66779000.00000000,"low_price":57312000.00000000,"trade_price":57391000.00000000,"timestamp":1705276799982,"candle_acc_trade_price":4504932294993.63827000,"candle_acc_trade_volume":72889.24232983,"first_day_of_period":"2024-01-08"},{"market":"KRW-BTC","candle_date_time_utc":"2024-01-01T00:00:00","candle_date_time_kst":"2024-01-01T09:00:00","opening_price":57045000.00000000,"high_price":60997000.00000000,"low_price":54346000.00000000,"trade_price":591080

- url의 끝에 `&count=200`으로 바뀐 것을 볼 수 있습니다
- 이번에도 json 형식으로 데이터를 전달 받았습니다
- 다시 DataFrame 형태로 바꿔봅시다

In [16]:
btc_df = pd.DataFrame(json.loads(response.text))
btc_df

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,first_day_of_period
0,KRW-BTC,2024-01-15T00:00:00,2024-01-15T09:00:00,57380000.0,59000000.0,57332000.0,58620000.0,1705339494255,1.948534e+11,3338.729270,2024-01-15
1,KRW-BTC,2024-01-08T00:00:00,2024-01-08T09:00:00,59105000.0,66779000.0,57312000.0,57391000.0,1705276799982,4.504932e+12,72889.242330,2024-01-08
2,KRW-BTC,2024-01-01T00:00:00,2024-01-01T09:00:00,57045000.0,60997000.0,54346000.0,59108000.0,1704671998330,2.615764e+12,44541.472695,2024-01-01
3,KRW-BTC,2023-12-25T00:00:00,2023-12-25T09:00:00,58002000.0,58659000.0,56000000.0,57047000.0,1704067199664,1.500536e+12,26230.477415,2023-12-25
4,KRW-BTC,2023-12-18T00:00:00,2023-12-18T09:00:00,56710000.0,60419000.0,55002000.0,58002000.0,1703462399990,2.082050e+12,35699.226661,2023-12-18
...,...,...,...,...,...,...,...,...,...,...,...
195,KRW-BTC,2020-04-20T00:00:00,2020-04-20T09:00:00,8660000.0,9377000.0,8300000.0,9376000.0,1587945602506,2.717172e+11,30664.426319,2020-04-20
196,KRW-BTC,2020-04-13T00:00:00,2020-04-13T09:00:00,8464000.0,8752000.0,7929000.0,8660000.0,1587340795956,2.544710e+11,30204.974181,2020-04-13
197,KRW-BTC,2020-04-06T00:00:00,2020-04-06T09:00:00,8198000.0,8957000.0,8183000.0,8470000.0,1586735995946,3.417039e+11,39654.926614,2020-04-06
198,KRW-BTC,2020-03-30T00:00:00,2020-03-30T09:00:00,7218000.0,8505000.0,7199000.0,8198000.0,1586131197225,3.954492e+11,49471.406794,2020-03-30


#### 과거 데이터 가져오기

- `Request Parameters`의 필드 중 `to`를 사용하면 마지막 캔들 시각을 조절 할 수 있다 합니다
- 설명 부분에서 `2023-01-01T00:00:00+09:00` 같은 형식으로 요청하라 합니다
- `QUERY PARAMS`의 `to`에 `2023-01-01T00:00:00+09:00`를 넣어봅니다
- 그리고 코드를 `복사 붙여넣기` 후 전달받은 json을 바로 DataFrame으로 바꿔봅니다

In [17]:
url = "https://api.upbit.com/v1/candles/weeks?market=KRW-BTC&to=2023-01-01T00%3A00%3A00%2B09%3A00&count=200"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

btc_df = pd.DataFrame(json.loads(response.text))
btc_df

,market,candle_date_time_utc,candle_date_time_kst,opening_price,high_price,low_price,trade_price,timestamp,candle_acc_trade_price,candle_acc_trade_volume,first_day_of_period
0,KRW-BTC,2022-12-26T00:00:00,2022-12-26T09:00:00,21854000.0,21861000.0,20700000.0,21123000.0,1672617597561,3.856169e+11,18153.280811,2022-12-26
1,KRW-BTC,2022-12-19T00:00:00,2022-12-19T09:00:00,22214000.0,22240000.0,21521000.0,21848000.0,1672012796861,3.565165e+11,16267.900330,2022-12-19
2,KRW-BTC,2022-12-12T00:00:00,2022-12-12T09:00:00,22784000.0,23941000.0,22000000.0,22210000.0,1671407999355,5.485700e+11,24039.625515,2022-12-12
3,KRW-BTC,2022-12-05T00:00:00,2022-12-05T09:00:00,22918000.0,23150000.0,22400000.0,22775000.0,1670803199985,4.055445e+11,17797.693636,2022-12-05
4,KRW-BTC,2022-11-28T00:00:00,2022-11-28T09:00:00,22661000.0,23400000.0,22155000.0,22906000.0,1670198397638,4.516840e+11,19831.443219,2022-11-28
...,...,...,...,...,...,...,...,...,...,...,...
195,KRW-BTC,2019-04-01T00:00:00,2019-04-01T09:00:00,4661000.0,5975000.0,4621000.0,5910000.0,1554681600018,6.246479e+11,115235.813522,2019-04-01
196,KRW-BTC,2019-03-25T00:00:00,2019-03-25T09:00:00,4526000.0,4680000.0,4420000.0,4661000.0,1554076797634,2.544972e+11,55830.276595,2019-03-25
197,KRW-BTC,2019-03-18T00:00:00,2019-03-18T09:00:00,4425000.0,4563000.0,4411000.0,4525000.0,1553471994092,2.167357e+11,48286.708089,2019-03-18
198,KRW-BTC,2019-03-11T00:00:00,2019-03-11T09:00:00,4336000.0,4492000.0,4269000.0,4425000.0,1552867190054,2.412776e+11,55497.999924,2019-03-11


- 더 과거의 데이터를 가져온 것을 확인하였습니다
- 해당 방법을 이용하여 지금보다 더 과거의 데이터를 가져올 수 있겠습니다